# Project – Phase 1 Group 07 Hugo Rocha

The database I chose was https://www.kaggle.com/nathanlauga/nba-games  <br>
Which is a database that was collected to work on NBA games data. <br><br>

The database has 5 datasets : <br>

<b>games.csv :</b> all games from 2004 season to last update with the date, teams and some details like number of points, etc. <br>
<b>games_details.csv :</b> details of games dataset, all statistics of players for a given game <br>
<b>players.csv :</b> players details (name)  <br>
<b>ranking.csv :</b> ranking of NBA given a day (split into west and east on CONFERENCE column <br>
<b>teams.csv :</b> all teams of NBA<br><br>
First I started working on the teams.csv file

## Imports
As for the usual imports I imported IRI baker which is a library that reliably creates valid (parts of) IRIs from strings (spaces are turned into underscores, etc.).


In [1]:
import csv
from csv import DictReader
from rdflib import Dataset, URIRef, Literal, Namespace, RDF, RDFS, OWL, XSD, Graph
from rdflib import Graph, URIRef, RDF, Literal
from rdflib.namespace import FOAF, XSD, RDFS
import pandas as pd
from iribaker import to_iri

## TEAMS 

TEAM_ID -> ID of the team <br>
TEAM_ID -> Minimum year of the team into NBA championship <br>
MIN_YEAR -> Maximum year of the team into NBA championship <br>
ABBREVIATION -> Abbreviation of team name<br>
NICKNAME -> Team's nickname<br>
YEARFOUNDED -> Founded Year<br>
CITY -> Team's city<br>
ARENA -> Team's stadium<br>
ARENACAPACITY -> Capacity of the stadium<br>

In [2]:
team_file = "teams.csv"
with open(team_file,'r') as csvfile:
    team_contents = [{k: v for k, v in row.items()}
        for row in csv.DictReader(csvfile, skipinitialspace=True, quotechar='"', delimiter=',')]

tms = Namespace("http://www.di.fc.ul.pt/~nba/team#")

graph=Graph()
graph.bind("tms", tms)

for s in team_contents:
    tid= URIRef(tms+s["TEAM_ID"])
    lid= URIRef(tms+s["LEAGUE_ID"])
    Min_year=Literal(s["MIN_YEAR"],datatype=XSD.int)
    Max_year=Literal(s["MAX_YEAR"],datatype=XSD.int)
    abre = Literal(s['ABBREVIATION'], datatype=XSD['string'])
    nick = URIRef(to_iri(tms+s["NICKNAME"]))
    nick_name =  Literal(s["NICKNAME"], lang='en')
    year_f=Literal(s["YEARFOUNDED"],datatype=XSD.int)
    City = URIRef(to_iri(tms+s["CITY"]))
    city_name =  Literal(s["CITY"], lang='en')
    Arena = URIRef(to_iri(tms+s["ARENA"]))
    arena_name =  Literal(s["ARENA"], lang='en')
    arenacapacity=Literal(s["ARENACAPACITY"],datatype=XSD.int)
  
    graph.add((tid, RDF.type, tms.Team))
    graph.add((tid, tms.team_id, tid))
    graph.add((tid, tms.league, lid))
    graph.add((tid, tms.min_year, Min_year))
    graph.add((tid, tms.max_year, Max_year))
    graph.add((tid, tms.abbreviation, abre))
    graph.add((tid, tms.nickname, nick))
    graph.add((nick, RDFS.label, nick_name))
    graph.add((tid, tms.year_found, year_f))
    graph.add((tid, tms.city, City))
    graph.add((City, RDFS.label, city_name))
    graph.add((tid, tms.arena, Arena))
    graph.add((Arena, RDFS.label, arena_name))
    graph.add((tid, tms.arena_capacity, arenacapacity))

## PLAYERS
PLAYER_NAME -> Player's name <br>
TEAM_ID -> ID of the team <br>
PLAYER_ID ->  ID of the player <br>
SEASON -> Season<br>

In [3]:
data = pd.read_csv('players.csv')

data["player_id"]  = data['PLAYER_ID'].apply(lambda x: str(x))
data["season"]  = data['SEASON'].apply(lambda x: str(x))
data["roster_id"] = data["player_id"]  + "_" + data["season"]
del data["player_id"]
del data["season"]
data.to_csv('players.csv')

pls = Namespace("http://www.di.fc.ul.pt/~nba/player#")
graph.bind("pls", pls)

team_file = "players.csv"
with open(team_file,'r') as csvfile:
    players_contents = [{k: v for k, v in row.items()}
        for row in csv.DictReader(csvfile, skipinitialspace=True, quotechar='"', delimiter=',')]

for p in players_contents:
    rid = URIRef(pls+p["roster_id"])
    pid = URIRef(pls+p["PLAYER_ID"])
    tid  = URIRef(tms+p["TEAM_ID"])
    Season=Literal(p["SEASON"],datatype=XSD.int)
    player = URIRef(to_iri(pls+p["PLAYER_NAME"]))
    player_name =  Literal(p["PLAYER_NAME"], lang='en')  
    
    graph.add((rid, RDF.type, pls.Player))
    graph.add((rid, pls.player_id, pid))
    graph.add((rid, tms.team, tid))
    graph.add((rid, pls.season, Season))
    graph.add((rid, pls.Name, player))
    graph.add((player, RDFS.label, player_name))

## Games
GAME_DATE_EST -> Game's date <br>
GAME_ID -> ID of the game  <br>
HOME_TEAM_ID -> ID of the home team <br>
VISITOR_TEAM_ID -> ID of the visitor team <br>
SEASON -> Season when the game occured <br>
PTS_home -> Number of points scored by home team <br> 
PTS_away -> Number of points scored by away team <br>

In [4]:
gms = Namespace("http://www.di.fc.ul.pt/~nba/games#")
graph.bind("gms", gms)

rank_file = "games.csv"
with open(rank_file,'r') as csvfile:
    game_contents = [{k: v for k, v in row.items()}
        for row in csv.DictReader(csvfile, skipinitialspace=True, quotechar='"', delimiter=',')]

     
for g in game_contents:
       
    game_id = URIRef(gms+g["GAME_ID"])  
    Game_Date=Literal(g["GAME_DATE_EST"],datatype=XSD.date)
    Season=Literal(g["SEASON"],datatype=XSD.int)
    Home= URIRef(tms+g["HOME_TEAM_ID"]) 
    Visitor= URIRef(tms+g["VISITOR_TEAM_ID"])
    points_home=Literal(g["PTS_home"],datatype=XSD.int)
    points_away=Literal(g["PTS_away"],datatype=XSD.int)
    
    graph.add((game_id, RDF.type, gms.Game))
    graph.add((game_id, gms.game_id, game_id))
    graph.add((game_id, gms.date, Game_Date))
    graph.add((game_id, gms.season, Season))
    graph.add((game_id, tms.home_team, Home))
    graph.add((game_id, tms.visitor_team, Visitor))
    graph.add((game_id, gms.Points_home, points_home))
    graph.add((game_id, gms.Points_away, points_away))
    

## Ranking
TEAM_ID -> ID of the team <br>
SEASON_ID -> Season<br>
STANDINGSDATE -> Standings date<br>
CONFERENCE -> Conference (west or east)<br>
TEAM -> Team name<br>
G -> Number of games played on the season<br>
W -> Number of winning games on the season<br>
L -> Number of loosing games on the season<br>
W_PCT -> Win %



In [5]:
data = pd.read_csv('ranking.csv')
data["team_idd"]  = data['TEAM_ID'].apply(lambda x: str(x))
data["date"]  = data['STANDINGSDATE'].apply(lambda x: str(x))
data["ranking_id"] = data["team_idd"]  + "_"+ data["date"]
del data["team_idd"]
del data["date"]
data.to_csv('ranking.csv')


rks = Namespace("http://www.di.fc.ul.pt/~nba/ranking#")
graph.bind("rks", rks)

rank_file = "ranking.csv"
with open(rank_file,'r') as csvfile:
    ranking_contents = [{k: v for k, v in row.items()}
        for row in csv.DictReader(csvfile, skipinitialspace=True, quotechar='"', delimiter=',')]


for r in ranking_contents:
       
    rank_id = URIRef(rks+r["ranking_id"])
    tid= URIRef(tms+r["TEAM_ID"]) 
    Season=Literal(r["SEASON_ID"],datatype=XSD.int)
    Date=Literal(r["STANDINGSDATE"],datatype=XSD.date)
    conference = URIRef(rks+r["CONFERENCE"])
    conference_name =  Literal(r["CONFERENCE"], lang='en')  
    team = URIRef(to_iri(rks+r["TEAM"]))
    team_name =  Literal(r["TEAM"], lang='en')  
    Games=Literal(r["G"],datatype=XSD.int)
    Wins=Literal(r["W"],datatype=XSD.int)
    Loses=Literal(r["L"],datatype=XSD.int)
    WL=Literal(r["W_PCT"],datatype=XSD.decimal)
    
    graph.add((rank_id, RDF.type, rks.Ranking))
    graph.add((rank_id, tms.team, tid))
    graph.add((rank_id, rks.season, Season))
    graph.add((rank_id, rks.date, Date))
    graph.add((rank_id, rks.Conference, conference))
    graph.add((conference, RDFS.label, conference_name))
    graph.add((rank_id, rks.Team, team))
    graph.add((team, RDFS.label, team_name))
    graph.add((rank_id, rks.games, Games))
    graph.add((rank_id, rks.wins, Wins))
    graph.add((rank_id, rks.loses, Loses))
    graph.add((rank_id, rks.W_PCT, WL))

## Game Details
o game_details.csv tem 75.36 MB o que demorou muito tempo a processar

In [6]:
gdt = Namespace("http://www.di.fc.ul.pt/~nba/games_details#")
graph.bind("gdt", gdt)

data = pd.read_csv('games_details.csv')

data["game_id"]  = data['GAME_ID'].apply(lambda x: str(x))
data["player_id"]  = data['PLAYER_ID'].apply(lambda x: str(x))
data["player_game"] = data["game_id"]  + "_" + data["player_id"]
del data["game_id"]
del data["player_id"]
data.to_csv('game_details.csv')

rank_file = "game_details.csv"
with open(rank_file,'r') as csvfile:
    game_details_contents = [{k: v for k, v in row.items()}
        for row in csv.DictReader(csvfile, skipinitialspace=True, quotechar='"', delimiter=',')]


for g in game_details_contents:
       
    player_game_id = URIRef(gdt+g["player_game"])
    gid= URIRef(gms+g["GAME_ID"]) 
    tid= URIRef(tms+g["TEAM_ID"]) 
    abre = Literal(g['TEAM_ABBREVIATION'], datatype=XSD['string'])
    City = URIRef(to_iri(tms+g["TEAM_CITY"]))
    city_name =  Literal(g["TEAM_CITY"], lang='en')
    pid= URIRef(pls+g["PLAYER_ID"]) 
    player = URIRef(to_iri(pls+g["PLAYER_NAME"]))
    player_name =  Literal(g["PLAYER_NAME"], lang='en') 
    
    graph.add((player_game_id, RDF.type, gdt.Game_Details))
    graph.add((player_game_id, gms.game, gid))
    graph.add((player_game_id, tms.team, tid))
    graph.add((player_game_id, tms.abbreviation, abre))
    graph.add((player_game_id, tms.city, City))
    graph.add((City, RDFS.label, city_name))
    graph.add((player_game_id, pls.player_id, pid))
    graph.add((player_game_id, pls.Name, player))
    graph.add((player, RDFS.label, player_name))

In [7]:
graph_data=graph.serialize(format='turtle')
fil=open("group07.ttl", "wb")
fil.write(graph_data)
fil.close()

# Sparql 
I run my queries in Apache Jena Fuseki here are the code and results
## Query 1
Em que equipas jogou o jogador Lebron James durante a sua carreira?


In [8]:
from SPARQLWrapper import SPARQLWrapper, XML, TSV



sparql = SPARQLWrapper("http://localhost:3030/FWS07/query")
query="""

prefix tm: <http://def.seegrid.csiro.au/isotc211/iso19108/2002/temporal#>
prefix gms: <http://www.di.fc.ul.pt/~nba/games#> 
prefix pls: <http://www.di.fc.ul.pt/~nba/player#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix rks: <http://www.di.fc.ul.pt/~nba/ranking#> 
prefix tms: <http://www.di.fc.ul.pt/~nba/team#> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 

SELECT ?season ?nickname
WHERE {
  ?p a tms:Team .
  ?p tms:team_id ?team .
  ?p tms:nickname ?nickname .
  ?s a pls:Player .
  ?s pls:Name pls:LeBron_James.
  ?s pls:season ?season .
  ?s tms:team ?team .
}
 ORDER BY ?season
       
"""
sparql.setQuery(query)

sparql.setReturnFormat(TSV)
results = sparql.query().convert().decode("utf-8")
print(results)

?season	?nickname
"2009"^^<http://www.w3.org/2001/XMLSchema#int>	<http://www.di.fc.ul.pt/~nba/team#Cavaliers>
"2010"^^<http://www.w3.org/2001/XMLSchema#int>	<http://www.di.fc.ul.pt/~nba/team#Heat>
"2011"^^<http://www.w3.org/2001/XMLSchema#int>	<http://www.di.fc.ul.pt/~nba/team#Heat>
"2012"^^<http://www.w3.org/2001/XMLSchema#int>	<http://www.di.fc.ul.pt/~nba/team#Heat>
"2013"^^<http://www.w3.org/2001/XMLSchema#int>	<http://www.di.fc.ul.pt/~nba/team#Heat>
"2014"^^<http://www.w3.org/2001/XMLSchema#int>	<http://www.di.fc.ul.pt/~nba/team#Cavaliers>
"2015"^^<http://www.w3.org/2001/XMLSchema#int>	<http://www.di.fc.ul.pt/~nba/team#Cavaliers>
"2016"^^<http://www.w3.org/2001/XMLSchema#int>	<http://www.di.fc.ul.pt/~nba/team#Cavaliers>
"2017"^^<http://www.w3.org/2001/XMLSchema#int>	<http://www.di.fc.ul.pt/~nba/team#Cavaliers>
"2018"^^<http://www.w3.org/2001/XMLSchema#int>	<http://www.di.fc.ul.pt/~nba/team#Lakers>
"2019"^^<http://www.w3.org/2001/XMLSchema#int>	<http://www.di.fc.ul.pt/~nba/team#Lake

## Query 2
Número de Vitórias da equipa em que o Lebron James jogava ao longo do mês de Dezembro de 2010



In [9]:
sparql = SPARQLWrapper("http://localhost:3030/FWS07/query")
query="""

PREFIX re: <http://www.w3.org/2000/10/swap/reason#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix tm: <http://def.seegrid.csiro.au/isotc211/iso19108/2002/temporal#>
prefix gms: <http://www.di.fc.ul.pt/~nba/games#> 
prefix pls: <http://www.di.fc.ul.pt/~nba/player#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix rks: <http://www.di.fc.ul.pt/~nba/ranking#> 
prefix tms: <http://www.di.fc.ul.pt/~nba/team#> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 


SELECT ?date ?w 
WHERE {

  ?p a tms:Team .
  ?p tms:team_id ?team .
  ?s a pls:Player .
  ?s pls:Name pls:LeBron_James.
  ?s pls:season ?se .
  ?s tms:team ?team .
  ?r a rks:Ranking .
  ?r tms:team ?team .
  ?r rks:date ?date . 
  ?r rks:wins ?w
  FILTER ( ?date >= "2010-12-01"^^xsd:date && ?date <= "2010-12-31"^^xsd:date  &&  ?se = 2010)
 }
ORDER BY ?date
       
"""
sparql.setQuery(query)

sparql.setReturnFormat(TSV)
results = sparql.query().convert().decode("utf-8")
print(results)

?date	?w
"2010-12-01"^^<http://www.w3.org/2001/XMLSchema#date>	"11"^^<http://www.w3.org/2001/XMLSchema#int>
"2010-12-02"^^<http://www.w3.org/2001/XMLSchema#date>	"12"^^<http://www.w3.org/2001/XMLSchema#int>
"2010-12-03"^^<http://www.w3.org/2001/XMLSchema#date>	"12"^^<http://www.w3.org/2001/XMLSchema#int>
"2010-12-04"^^<http://www.w3.org/2001/XMLSchema#date>	"13"^^<http://www.w3.org/2001/XMLSchema#int>
"2010-12-05"^^<http://www.w3.org/2001/XMLSchema#date>	"13"^^<http://www.w3.org/2001/XMLSchema#int>
"2010-12-06"^^<http://www.w3.org/2001/XMLSchema#date>	"14"^^<http://www.w3.org/2001/XMLSchema#int>
"2010-12-07"^^<http://www.w3.org/2001/XMLSchema#date>	"14"^^<http://www.w3.org/2001/XMLSchema#int>
"2010-12-08"^^<http://www.w3.org/2001/XMLSchema#date>	"15"^^<http://www.w3.org/2001/XMLSchema#int>
"2010-12-09"^^<http://www.w3.org/2001/XMLSchema#date>	"15"^^<http://www.w3.org/2001/XMLSchema#int>
"2010-12-10"^^<http://www.w3.org/2001/XMLSchema#date>	"16"^^<http://www.w3.org/2001/XMLSchema#int>
"

## Query 3
Todos os Jogos em que a equipa da casa marcou mais de 140 pontos




In [10]:
sparql = SPARQLWrapper("http://localhost:3030/FWS07/query")
query="""

PREFIX tm: <http://def.seegrid.csiro.au/isotc211/iso19108/2002/temporal#>
PREFIX gm: <http://def.seegrid.csiro.au/isotc211/iso19107/2003/geometry#>
prefix gms: <http://www.di.fc.ul.pt/~nba/games#> 
prefix pls: <http://www.di.fc.ul.pt/~nba/player#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix rks: <http://www.di.fc.ul.pt/~nba/ranking#> 
prefix tms: <http://www.di.fc.ul.pt/~nba/team#> 
prefix xsd: <http://www.w3.org/2001/XMLSchema#> 

SELECT ?team ?date ?points 
WHERE {
  
  ?game a gms:Game .
  ?game gms:Points_home ?points .
  ?game gms:date ?date .
  ?game tms:home_team ?l .
  ?t a tms:Team .
  ?t tms:team_id ?l .
  ?t tms:nickname ?team .
  FILTER (?points > 140) 
       }
  ORDER BY ?points  
       
"""
sparql.setQuery(query)

sparql.setReturnFormat(TSV)
results = sparql.query().convert().decode("utf-8")
print(results)

?team	?date	?points
<http://www.di.fc.ul.pt/~nba/team#Celtics>	"2020-02-13"^^<http://www.w3.org/2001/XMLSchema#date>	"141"^^<http://www.w3.org/2001/XMLSchema#int>
<http://www.di.fc.ul.pt/~nba/team#Hawks>	"2020-02-28"^^<http://www.w3.org/2001/XMLSchema#date>	"141"^^<http://www.w3.org/2001/XMLSchema#int>
<http://www.di.fc.ul.pt/~nba/team#Timberwolves>	"2020-02-08"^^<http://www.w3.org/2001/XMLSchema#date>	"142"^^<http://www.w3.org/2001/XMLSchema#int>

